In [1]:
import pandas as pd
import numpy as np
import os
import sys

# --- 0. 경로 설정 ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(PROJECT_ROOT, "data", "processed")
FINAL_MASTER_FILE = os.path.join(DATA_DIR, "final_master_table_v2.csv")

# --- 1. DataHandler 클래스 정의 ---

class DataHandler:
    def __init__(self, file_path):
        self.file_path = file_path
        self.data_by_ticker = {}
        self.tickers = []
        self._load_and_process_data()
        
    def _load_and_process_data(self):
        try:
            # 1. 'ticker'를 'str'로 읽어옴
            df = pd.read_csv(
                self.file_path, 
                parse_dates=['date'],
                dtype={'ticker': str} 
            )
            
            # [!!! 최종 수정 !!!]
            # 2. '0'이 누락된 문자열을 6자리로 채웁니다 (zero-fill)
            #    '10140' -> '010140'
            #    '9540'  -> '009540'
            df['ticker'] = df['ticker'].str.zfill(6)
            
            df = df.set_index('date')
            
            # 3. 이제 self.tickers에는 '010140', '009540' 등이 담깁니다.
            self.tickers = df['ticker'].unique()
            
            for ticker in self.tickers:
                ticker_df = df[df['ticker'] == ticker].copy()
                channel_cols = [col for col in ticker_df.columns if col not in ['ticker']]
                self.data_by_ticker[ticker] = ticker_df[channel_cols]
            
            print(f"[DataHandler] Success: Loaded and processed data for {len(self.tickers)} tickers.")
            print(f"[DataHandler] Available tickers: {self.tickers}") # <- '0'이 살아났는지 확인!

        except FileNotFoundError:
            print(f"[DataHandler] Error: File not found at {self.file_path}")
        except Exception as e:
            print(f"[DataHandler] Error loading data: {e}")

    def get_data_by_ticker(self, ticker):
        if ticker not in self.data_by_ticker:
            print(f"[DataHandler] Error: No data found for ticker {ticker}")
            return None
        return self.data_by_ticker[ticker]

    def get_all_tickers(self):
        return self.tickers

# --- 2. DataHandler 클래스 인스턴스화 및 테스트 ---
print(f"Loading data from: {FINAL_MASTER_FILE}")


Loading data from: /workspace/ship-ai/data/processed/final_master_table_v2.csv


In [3]:

# [1] 커널 재시작 후, [2] 위 셀을 먼저 실행하고, [3] 이 셀을 실행
data_handler = DataHandler(FINAL_MASTER_FILE)

# 테스트: '삼성중공업' (010140) 데이터 가져오기
# 이제 if '010140' in ['010140', '009540', ...] 비교가 True가 됩니다.
if '010140' in data_handler.get_all_tickers():
    print("\n--- Test: Get '010140' (삼성중공업) Data ---")
    samsung_df = data_handler.get_data_by_ticker('010140')
    if samsung_df is not None:
        print(samsung_df.head())
        print(f"\nShape of data: {samsung_df.shape}")
        print(f"Columns (12 channels): {list(samsung_df.columns)}")

[DataHandler] Success: Loaded and processed data for 6 tickers.
[DataHandler] Available tickers: ['010140' '010620' '329180' '042660' '443060' '009540']

--- Test: Get '010140' (삼성중공업) Data ---
            close_log    ret_1d  trading_volume_log       roe  \
date                                                            
2019-05-15   8.922125  0.016545           14.907388 -1.539182   
2019-05-16   8.900413 -0.021712           14.858667 -1.539182   
2019-05-17   8.884749 -0.015664           15.513969 -1.539182   
2019-05-20   8.871786 -0.012963           14.881691 -1.539182   
2019-05-21   8.882253  0.010467           14.772045 -1.539182   

            real_debt_ratio  new_order_event_impulse  new_order_count_stair  \
date                                                                          
2019-05-15       119.929095                      0.0                    1.0   
2019-05-16       119.929095                      0.0                    1.0   
2019-05-17       119.929095       

## Phase2-B : 윈도우 생성기 구현
LLM에 넣기 위해 슬라이딩 윈도우 방식의 3D 데이터로 재구성합니다.
<br>(샘플 수, 120일, 12개 채널) 형태의 numpy 배열(텐서)을 만듭니다.

In [4]:
import numpy as np
import pandas as pd

# --- 1. 윈도우 생성기 함수 정의 ---

def create_sliding_windows(data, input_seq_len, output_seq_len):
    """
    DataFrame(2D)을 받아 슬라이딩 윈도우(3D) numpy 배열로 변환합니다.
    
    Args:
        data (pd.DataFrame): 12개 채널이 포함된 2D 데이터프레임
        input_seq_len (int): 모델이 입력을 위해 볼 과거 날짜 (예: 120일)
        output_seq_len (int): 모델이 예측할 미래 날짜 (예: 10일)
        
    Returns:
        (np.array, np.array): (X: 입력 윈도우, y: 타겟 윈도우) 
                               X shape: (N, 120, 12), y shape: (N, 10, 12)
    """
    
    # DataFrame을 numpy 배열로 변환
    data_np = data.values
    n_samples = len(data_np)
    
    X = [] # 입력 (과거 120일)
    y = [] # 타겟 (미래 10일)
    
    total_len = input_seq_len + output_seq_len
    
    # 윈도우를 한 칸씩 이동(slide)하며 데이터 추출
    for i in range(n_samples - total_len + 1):
        input_window = data_np[i : i + input_seq_len]
        output_window = data_np[i + input_seq_len : i + total_len]
        
        X.append(input_window)
        y.append(output_window)
        
    return np.array(X), np.array(y)

# --- 2. 윈도우 생성기 테스트 ---

# (이전 셀에서 'data_handler'가 이미 생성되었다고 가정)
if 'data_handler' in locals():
    print("--- Test: Creating Sliding Windows for '010140' ---")
    
    # 1. 삼성중공업(010140) 데이터 가져오기
    samsung_df = data_handler.get_data_by_ticker('010140')
    
    if samsung_df is not None:
        # 2. 모델 하이퍼파라미터 정의
        INPUT_SEQ_LEN = 120  # 입력 시퀀스 길이 (약 6개월)
        OUTPUT_SEQ_LEN = 10  # 예측 시퀀스 길이 (2주)
        
        print(f"Original data shape (2D): {samsung_df.shape}")
        print(f"Input sequence length: {INPUT_SEQ_LEN} days")
        print(f"Output sequence length: {OUTPUT_SEQ_LEN} days")

        # 3. 윈도우 생성 함수 호출
        X_samsung, y_samsung = create_sliding_windows(
            samsung_df, 
            INPUT_SEQ_LEN, 
            OUTPUT_SEQ_LEN
        )
        
        print("\n[SUCCESS] Sliding windows created!")
        print(f"  > X (Input Tensors) shape (3D): {X_samsung.shape}")
        print(f"  > y (Target Tensors) shape (3D): {y_samsung.shape}")
        
        # 예: X_samsung.shape이 (약 1200, 120, 12) 처럼 나오면 성공
        
    else:
        print("[Error] '010140' data is None.")
else:
    print("[Error] 'data_handler' is not defined. Please run Phase 2-A cell first.")

--- Test: Creating Sliding Windows for '010140' ---
Original data shape (2D): (1388, 12)
Input sequence length: 120 days
Output sequence length: 10 days

[SUCCESS] Sliding windows created!
  > X (Input Tensors) shape (3D): (1259, 120, 12)
  > y (Target Tensors) shape (3D): (1259, 10, 12)


## Phase2-B : 윈도우 생성기 구현
12개 채널의 Scale 일치를 위해 Standardization을 진행합니다.
<br>모든 12개 채널의 Scale을평균 0, 표준편차 1인 표준 정규 분포로 통일시킵니다.

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.preprocessing import StandardScaler

# --- 0. 경로 설정 ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(PROJECT_ROOT, "data", "processed")
FINAL_MASTER_FILE = os.path.join(DATA_DIR, "final_master_table_v2.csv")

# --- 1. [업그레이드된] DataHandler 클래스 정의 ---

class DataHandler:
    """
    [V2] 표준화(Standardization) 기능이 추가된 DataHandler.
    """
    
    def __init__(self, file_path, train_end_date='2022-12-31'):
        """
        [수정] train_end_date를 받아 Scaler를 학습시킵니다.
        """
        self.file_path = file_path
        self.train_end_date = pd.to_datetime(train_end_date)
        self.data_by_ticker = {}   # 원본 데이터 보관
        self.scalers_by_ticker = {} # Ticker별 Scaler 보관
        self.tickers = []
        
        self._load_and_process_data()
        self._fit_scalers() # Scaler 학습 단계 추가
        
    def _load_and_process_data(self):
        """
        [수정] 'ticker'를 zfill(6)로 채워서 로드합니다.
        """
        try:
            df = pd.read_csv(
                self.file_path, 
                parse_dates=['date'],
                dtype={'ticker': str} 
            )
            df['ticker'] = df['ticker'].str.zfill(6)
            df = df.set_index('date')
            
            self.tickers = df['ticker'].unique()
            
            for ticker in self.tickers:
                ticker_df = df[df['ticker'] == ticker].copy()
                channel_cols = [col for col in ticker_df.columns if col not in ['ticker']]
                # [수정] 원본 데이터를 data_by_ticker에 저장
                self.data_by_ticker[ticker] = ticker_df[channel_cols]
            
            print(f"[DataHandler V2] Success: Loaded and processed data for {len(self.tickers)} tickers.")
            print(f"[DataHandler V2] Available tickers: {self.tickers}")

        except FileNotFoundError:
            print(f"[DataHandler V2] Error: File not found at {self.file_path}")
        except Exception as e:
            print(f"[DataHandler V2] Error loading data: {e}")

    def _fit_scalers(self):
        """
        [!!! NEW !!!]
        데이터 리크(Leakage) 방지를 위해 'train_end_date' 이전 데이터로만
        StandardScaler를 학습(fit)시킵니다.
        """
        print(f"[DataHandler V2] Fitting scalers using data up to {self.train_end_date.date()}...")
        for ticker in self.tickers:
            # 1. 훈련 데이터 분리 (예: ~ 2022-12-31)
            train_data = self.data_by_ticker[ticker].loc[:self.train_end_date]
            
            if train_data.empty:
                print(f"  > Warning: No training data for {ticker} before {self.train_end_date.date()}. Skipping scaler.")
                continue
                
            # 2. 12개 채널에 대한 Scaler 생성 및 학습
            scaler = StandardScaler()
            scaler.fit(train_data) # [중요] 'fit'은 훈련 데이터로만!
            
            # 3. Ticker별로 학습된 Scaler 저장
            self.scalers_by_ticker[ticker] = scaler
        print("[DataHandler V2] Scalers fitted.")

    def get_scaled_data_by_ticker(self, ticker):
        """
        [!!! NEW !!!]
        원본 데이터를 Ticker에 맞는 Scaler로 'transform'하여 반환합니다.
        """
        if ticker not in self.data_by_ticker:
            print(f"[DataHandler V2] Error: No data found for ticker {ticker}")
            return None
        if ticker not in self.scalers_by_ticker:
            print(f"[DataHandler V2] Error: No scaler found for ticker {ticker}")
            return None

        # 1. 원본 데이터 전체를 가져옴
        original_data = self.data_by_ticker[ticker]
        
        # 2. "학습된" Scaler로 "전체" 데이터를 변환(transform)
        scaled_data_np = self.scalers_by_ticker[ticker].transform(original_data)
        
        # 3. 다시 DataFrame으로 변환 (인덱스와 컬럼명 복원)
        scaled_df = pd.DataFrame(
            scaled_data_np, 
            index=original_data.index, 
            columns=original_data.columns
        )
        return scaled_df

    def get_all_tickers(self):
        return self.tickers

# --- 2. [업그레이드된] DataHandler 테스트 ---
print(f"Loading data from: {FINAL_MASTER_FILE}")

# [1] 커널 재시작 후, [2] 위 셀을 먼저 실행하고, [3] 이 셀을 실행
# (훈련 데이터는 2022년 말까지로 설정)
data_handler_v2 = DataHandler(FINAL_MASTER_FILE, train_end_date='2022-12-31')

# 테스트: '삼성중공업' (010140)의 "표준화된" 데이터 가져오기
if '010140' in data_handler_v2.get_all_tickers():
    print("\n--- Test: Get '010140' (삼성중공업) SCALED Data ---")
    
    scaled_samsung_df = data_handler_v2.get_scaled_data_by_ticker('010140')
    
    if scaled_samsung_df is not None:
        print(scaled_samsung_df.head())
        print(f"\nShape of scaled data: {scaled_samsung_df.shape}")
        
        # [검증] 표준화가 잘 되었는지 평균/표준편차 확인
        print("\n--- Verification (Scaled Data Stats) ---")
        print(" (평균은 0에 가깝고, 표준편차는 1에 가까워야 함)")
        print(scaled_samsung_df.describe().loc[['mean', 'std']])